In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

df_full = pd.read_csv("/kaggle/input/hipparcos-star-catalog/hipparcos-voidmain.csv")
print(df_full.columns)
df = df_full[df_full["Plx"]>0]
PAR = np.array(df["Plx"],dtype=float)
# note there are columns in the database that already have RA and DEC in degrees
RA = np.array(df["RAdeg"],dtype=float)
DEC = np.array(df["DEdeg"],dtype=float)
m = np.array(df["Vmag"],dtype=float)
bmv = np.array(df["B-V"],dtype=float)
RA *= np.pi/180 #convert to radians
DEC *= np.pi/180


get cartesian corrdinates from parallax, RA, and DEC

In [ ]:
# Anaconda code 
Distance = 1/PAR*1000   #the 1000 is because parallax in database is in milli arcseconds

Phi = (np.pi/2-DEC)
Theta = RA

x = Distance*np.cos(Theta)*np.sin(Phi)
y = Distance*np.sin(Theta)*np.sin(Phi)
z = Distance*np.cos(Phi)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x,y,z,'.')
plt.show()

In [ ]:
# get x y and z into a single array for processing
xyz = np.transpose(np.array([x,y,z]))


In [ ]:
# DBSCN clustering algorithm 

from sklearn.cluster import DBSCAN
db=DBSCAN(eps=2,min_samples=4,metric='euclidean')


In [ ]:
model=db.fit(xyz)

In [ ]:
label=model.labels_

In [ ]:
df = pd.DataFrame({"RA":RA,"DEC":DEC,"label":label,"bmv":bmv})
filtered = df[df["label"]>-1]
import plotly.express as px
fig = px.scatter(filtered,x="RA",y="DEC",color="bmv")
fig.show()

In [ ]:

import plotly.express as px
fig = px.scatter(df,x="RA",y="DEC",color="label")
fig.show()

In [ ]:
from sklearn import metrics

#identifying the points which makes up our core points
sample_cores=np.zeros_like(label,dtype=bool)

sample_cores[db.core_sample_indices_]=True

#Calculating the number of clusters

n_clusters=len(set(label))- (1 if -1 in label else 0)
print('No of clusters:',n_clusters)


In [ ]:
y_means = db.fit_predict(x)
plt.figure(figsize=(7,5))
plt.scatter(x[y_means == 0, 0], x[y_means == 0, 1], s = 50, c = 'pink')
plt.scatter(x[y_means == 1, 0], x[y_means == 1, 1], s = 50, c = 'yellow')
plt.scatter(x[y_means == 2, 0], x[y_means == 2, 1], s = 50, c = 'cyan')
plt.scatter(x[y_means == 3, 0], x[y_means == 3, 1], s = 50, c = 'magenta')
plt.scatter(x[y_means == 4, 0], x[y_means == 4, 1], s = 50, c = 'orange')
plt.scatter(x[y_means == 5, 0], x[y_means == 5, 1], s = 50, c = 'blue')
plt.scatter(x[y_means == 6, 0], x[y_means == 6, 1], s = 50, c = 'red')
plt.scatter(x[y_means == 7, 0], x[y_means == 7, 1], s = 50, c = 'black')
plt.scatter(x[y_means == 8, 0], x[y_means == 8, 1], s = 50, c = 'violet')
plt.xlabel('Annual Income in (1k)')
plt.ylabel('Spending Score from 1-100')
plt.title('Clusters of Data')
plt.show()

Hierarchial Clustering

In [ ]:
import scipy.cluster.hierarchy as sch

dendrogram = sch.dendrogram(sch.linkage(x, method = 'ward'))
plt.title('Dendrogam', fontsize = 20)
plt.xlabel('Foo')
plt.ylabel('Foo')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

hc = AgglomerativeClustering(n_clusters = 9, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(x)

plt.scatter(x[y_hc == 0, 0], x[y_hc == 0, 1], s = 50, c = 'pink')
plt.scatter(x[y_hc == 1, 0], x[y_hc == 1, 1], s = 50, c = 'yellow')
plt.scatter(x[y_hc == 2, 0], x[y_hc == 2, 1], s = 50, c = 'cyan')
plt.scatter(x[y_hc == 3, 0], x[y_hc == 3, 1], s = 50, c = 'magenta')
plt.scatter(x[y_hc == 4, 0], x[y_hc == 4, 1], s = 50, c = 'orange')
plt.scatter(x[y_hc == 5, 0], x[y_hc == 5, 1], s = 50, c = 'blue')
plt.scatter(x[y_hc == 6, 0], x[y_hc == 6, 1], s = 50, c = 'red')
plt.scatter(x[y_hc == 7, 0], x[y_hc == 7, 1], s = 50, c = 'black')
plt.scatter(x[y_hc == 8, 0], x[y_hc == 8, 1], s = 50, c = 'violet')


plt.title('Hierarchial Clustering', fontsize = 20)
plt.xlabel('Foo')
plt.ylabel('Foo')
plt.legend()
plt.grid()
plt.show()